Write a series of functions to estimate the value function using approximation. The idea here is to guess some coefficients (for the basis functions) and in each iteration the model will imply another set of coefficients and you do that until everything converges. 


# Step 1: Specify the initial guess and the interval of capital that we are trying to maximize on 

In [7]:
 using LinearAlgebra
 using Optim
 using Plots
 params = (alpha = 0.75, # capital share
           beta = 0.95, # discount
           eta = 2, # EMUC
           steady_state = (0.75*0.95)^(1/(1 - 0.75)),
           k_0 = (0.75*0.95)^(1/(1 - 0.75))/2, # initial state
           capital_upper = (0.75*0.95)^(1/(1 - 0.75))*1.01, # upper bound
           capital_lower = (0.75*0.95)^(1/(1 - 0.75))/2, # lower bound
           num_points = 7, # number of grid points
           tolerance = 0.0001)

(alpha = 0.75, beta = 0.95, eta = 2, steady_state = 0.25771486816406236, k_0 = 0.12885743408203118, capital_upper = 0.26029201684570297, capital_lower = 0.12885743408203118, num_points = 7, tolerance = 0.0001)

# Step 2: Make the guess of the betas (coefficients on the basis functions)

In [8]:
 coefficients = zeros(params.num_points) # # coeffs = # grid points in collocation

7-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

# Step 3: Select the convergence rule:

Max change in value on the grid is < 0.0001%

# Step 4: Construct grid points

In [10]:
function cheb_polys(x, n)
     if n == 0
         return x ./ x               # T_0(x) = 1
     elseif n == 1
         return x                    # T_1(x) = x
     else
         cheb_recursion(x, n) =
             2x .* cheb_polys.(x, n-1) .- cheb_polys.(x, n-2)
         return cheb_recursion(x, n) # T_n(x) = 2xT_{n-1}(x) - T_{n-2}(x)
     end
 end;

In [11]:
cheb_nodes(n) = cos.(pi * (2*(1:n) .- 1)./(2n));
grid = cheb_nodes(params.num_points) # [-1, 1] grid with n points

7-element Vector{Float64}:
  0.9749279121818236
  0.7818314824680298
  0.4338837391175582
  6.123233995736766e-17
 -0.43388373911755806
 -0.7818314824680297
 -0.9749279121818236

Need to "scale" the grid from [-1,1] to be the interval that we have and "descale"

In [12]:
expand_grid(grid, params) = # function that expands [-1,1] to [a,b]
(1 .+ grid)*(params.capital_upper - params.capital_lower)/2 .+ params.capital_lower
capital_grid = expand_grid(grid, params)

7-element Vector{Float64}:
 0.2586443471450049
 0.2459545728087113
 0.2230883895732961
 0.19457472546386706
 0.16606106135443804
 0.14319487811902284
 0.13050510378272925

In [13]:
 shrink_grid(capital) = 
   2*(capital - params.capital_lower)/(params.capital_upper - params.capital_lower) - 1;
 shrink_grid.(capital_grid)

7-element Vector{Float64}:
  0.9749279121818237
  0.7818314824680297
  0.43388373911755806
 -2.220446049250313e-16
 -0.43388373911755806
 -0.7818314824680297
 -0.9749279121818236

For each grid point, evaluate the n degree chebyshev polynomials.

In [14]:
 construct_basis_matrix(grid, params) = hcat([cheb_polys.(shrink_grid.(grid), n) for n = 0:params.num_points - 1]...);
 basis_matrix = construct_basis_matrix(capital_grid, params)
 basis_inverse = basis_matrix \ I # pre-invert


7×7 Matrix{Float64}:
 0.142857    0.142857    0.142857   …   0.142857    0.142857    0.142857
 0.278551    0.22338     0.123967      -0.123967   -0.22338    -0.278551
 0.25742     0.0635774  -0.17814       -0.17814     0.0635774   0.25742
 0.22338    -0.123967   -0.278551       0.278551    0.123967   -0.22338
 0.17814    -0.25742    -0.0635774     -0.0635774  -0.25742     0.17814
 0.123967   -0.278551    0.22338    …  -0.22338     0.278551   -0.123967
 0.0635774  -0.17814     0.25742        0.25742    -0.17814     0.0635774

After having the basis matrix, multiply it by the coefficients to get the value functions at every grid point

In [15]:
 eval_value_function(coefficients, grid, params) = construct_basis_matrix(grid, params) * coefficients;
eval_value_function([1,1,1,1,1,1,1], grid, params)

7-element Vector{Float64}:
     9.26624982788923e7
     1.6940689772371337e7
 77224.00563563094
 15354.384388936067
     2.287632997495987e7
     3.3075659788769543e8
     9.839582890300741e8

In [16]:
grid

7-element Vector{Float64}:
  0.9749279121818236
  0.7818314824680298
  0.4338837391175582
  6.123233995736766e-17
 -0.43388373911755806
 -0.7818314824680297
 -0.9749279121818236

In [17]:
function loop_grid(params, capital_grid, coefficients)
     max_value = similar(coefficients); # initialized max value vector
     # Inner loop over grid points
     for (iteration, capital) in enumerate(capital_grid)
         # Define Bellman as a closure
         function bellman(consumption)
             capital_next = capital^params.alpha - consumption # Next period state
             cont_value = eval_value_function(coefficients, capital_next, params)[1] # Continuation value
             value_out = (consumption)^(1-params.eta)/(1-params.eta) + params.beta*cont_value # Utility + continuation value
             return -value_out
         end;
         results = optimize(bellman, 0.00*capital^params.alpha, 0.99*capital^params.alpha) # maximize Bellman
         max_value[iteration] = -Optim.minimum(results) # Store max value in vector
     end
     return max_value
 end


loop_grid (generic function with 1 method)

In [18]:
function solve_vfi(params, basis_inverse, capital_grid, coefficients)
     iteration = 1
     error = 1e10;
     max_value = similar(coefficients);
     value_prev = .1*ones(params.num_points);
     coefficients_store = Vector{Vector}(undef, 1)
     coefficients_store[1] = coefficients
     while error > params.tolerance # Outer loop iterating on Bellman eq
         max_value = loop_grid(params, capital_grid, coefficients) # Inner loop
         coefficients = basis_inverse*max_value # \Psi \ y, recover coefficients
         error = maximum(abs.((max_value - value_prev)./(value_prev))) # compute error
         value_prev = deepcopy(max_value) # save previous values
         if mod(iteration, 5) == 0
             println("Maximum Error of $(error) on iteration $(iteration).")
             append!(coefficients_store, [coefficients])
         end
         iteration += 1
     end
     return coefficients, max_value, coefficients_store
 end

solve_vfi (generic function with 1 method)

In [19]:
 solution_coeffs, max_value, intermediate_coefficients =
     solve_vfi(params, basis_inverse, capital_grid, coefficients)

Maximum Error of 0.330191988422609 on iteration 5.
Maximum Error of 0.1080139919745117 on iteration 10.
Maximum Error of 0.056479178550114986 on iteration 15.
Maximum Error of 0.03483338924508334 on iteration 20.
Maximum Error of 0.023286433761111964 on iteration 25.
Maximum Error of 0.01630154309258161 on iteration 30.
Maximum Error of 0.01174748047041362 on iteration 35.
Maximum Error of 0.008631245645920499 on iteration 40.
Maximum Error of 0.006427690604126993 on iteration 45.
Maximum Error of 0.00483307368424568 on iteration 50.
Maximum Error of 0.0036597148900624076 on iteration 55.
Maximum Error of 0.0027856923769764997 on iteration 60.
Maximum Error of 0.002128686710212966 on iteration 65.
Maximum Error of 0.0016314249677368766 on iteration 70.
Maximum Error of 0.00125311605713912 on iteration 75.
Maximum Error of 0.0009641708791271023 on iteration 80.
Maximum Error of 0.0007428166750767919 on iteration 85.
Maximum Error of 0.0005728521498806165 on iteration 90.
Maximum Error o

([-200.6291758538632, 9.991472391067632, -1.2278992641150808, 0.1737946046008787, -0.02621191019446396, 0.003954006913225118, -0.0007409750421140403], [-191.87342361439306, -193.14594489252323, -195.68963652528424, -199.42674752490046, -204.0272196380861, -208.61071718644214, -211.63054159541304], Vector[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-36.85894901060744, 6.259861237095086, -0.8221318972861216, 0.11882022830524086, -0.01784945487786284, 0.002692033153908532, -0.00037520235133792955], [-74.87645132736961, 8.931829874526969, -1.123822713129858, 0.16080476492852824, -0.02429279880713139, 0.0039027323348115577, -0.0006447123917356167], [-103.80959945589589, 9.689013116843583, -1.1994026067805663, 0.170560955466307, -0.025548140549795404, 0.003969430583560096, -0.0007161344250570962], [-125.92003087539777, 9.90440331937696, -1.219552059022952, 0.17300404781583012, -0.02601005829928537, 0.003959590376937803, -0.0007341995822632486], [-142.9397465762615, 9.966570886453816, -1.2253830534

# FIXED POINT ITERATION

All the steps are very similar to VFI, the only difference is when we run the inner loop, replace maximization of the bellman equation with the euler equation

In [20]:
 eval_policy_function(coefficients, capital, params_fpi) = 
     construct_basis_matrix(capital, params_fpi) * coefficients

eval_policy_function (generic function with 1 method)

In [21]:
function cons_euler(params, capital, coeffs)
    # get the arguments of the equations
    consumption = eval_policy_function(coeffs, capital, params)
    capital_next = capital^params.alpha - consumption
    consumption_next = eval_policy_function(coeffs, capital_next, params)
    
    # plug in
    f_prime = params.alpha * capital_next^(params.alpha - 1)
    u_prime = consumption^(-params.eta)
    result = (params.beta*u_prime*f_prime)^(-1/eta)
    return result
end

cons_euler (generic function with 1 method)